In [4]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import get_dataset_dir

from collections import defaultdict



def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rawid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rawid_to_name[line[0]] = (line[1], line[2])

    return rawid_to_name

# Input user number
input_user_id = input('Enter user number: ')

# First, train the algortihm to compute the similarities between users
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': True, 'min_support': 5}
algo = KNNWithMeans(k=4, min_k=4, sim_options=sim_options)
algo.fit(trainset)

# Drop out the missing ratings and make predictions
testset = trainset.build_anti_testset()
testset = filter(lambda x: x[0] == input_user_id, testset)
predictions = algo.test(testset)


# Create for each iser list of predictions for items
top_n = defaultdict(list)
for uid, iid, _, est, _ in predictions:
    top_n[uid].append((iid, round(est, 3)))

# For each user sort items and leave only 5 most valid
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:5]

# Get item names
item_names = read_item_names()

best_films_names = []

for movie_rid, rating in top_n[input_user_id]:
    best_films_names.append(item_names[movie_rid][0])
    
print("Наиболее подходящие фильмы:", best_films_names)
    
# Print results
#print(f'User {input_user_id}:')
#for movie_rid, rating in top_n[input_user_id]:
#    print('{:4s} {:<60s} {}'.format(movie_rid, str(item_names[movie_rid]), rating))

Enter user number: 6
Computing the cosine similarity matrix...
Done computing similarity matrix.
Наиболее подходящие фильмы: ['World of Apu, The (Apur Sansar) (1959)', 'Faust (1994)', 'Pather Panchali (1955)', 'Dead Man (1995)', 'Titanic (1997)']


In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import pandas as pd
import requests
import re

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

In [6]:
def Sparql(sparql, film):
    print(film)
# Список фильмов, которые получили Оскар или награду на Каннском фестивале в том же году, что и заданный фильм
    queryString = """
# Запрос 11 - список фильмов, которые получили Оскар или награду на Каннском фестивале в том же году
# Фильм 3   - The Godfather: Part II

# Ставлю DISTINCT, т.к. некоторые фильмы получили более одной премии единовременно
SELECT DISTINCT ?movieLabel ?year  WHERE {
  
  # С учетом, что награда - это "Оскар" или награда Каннского фестиваля
  { ?award wdt:P31 wd:Q19020 . } UNION { ?award wdt:P31 wd:Q28444913 . }

  # Находим уникальные годы выдачи Оскаров и наград Каннского фестиваля фильму 'The Godfather: Part II'
  # Ставлю DISTINCT, т.к. фильм получил более одной премии единовременно
  { 
    SELECT DISTINCT ?year
    WHERE {
      wd:"""+film+""" p:P166 ?awardStat.          # Получаем все регалии фильма "Крестный отец"...
      ?awardStat ps:P166 ?award.             # ... которые получены как награды выбранного типа
      ?awardStat pq:P805 ?awardEdition .     # Получаем событие выдачи премии
      ?awardEdition wdt:P585 ?date .         # Получаем дату события
      BIND(str(YEAR(?date)) AS ?year). }  # Извлекаем номер года
  }
  
  ?movie wdt:P31 wd:Q11424.                  # Выбираем фильмы...
  ?movie p:P166 ?awardStat.                  # ... имеющие регалии...
  ?awardStat ps:P166 ?award.                 # ... которые получены как награды выбранного типа
  ?awardStat pq:P585 ?date.                  # Получаем дату вручения
  FILTER(str(YEAR(?date)) = ?year).        # Проверяем совпадение года
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
limit 100
    """
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df

In [8]:
print("Список фильмов, которые получили Оскар или награду на Каннском фестивале в том же году, что и заданный фильм")
# выводим результаты запроса для каждого фильма в списке рекомендаций
for f in best_films_names:
    print(f+":")
    film = re.sub(' \([0-9]{4}\)', "", f) # убираем год
    params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': film}
    res = requests.get(API_ENDPOINT, params = params)
    #проверяем, нашелся ли фильм
    if res.json()['search']:
        #если фильм найден, то запускаем выполнение запроса
        results_df = Sparql(sparql, res.json()['search'][0]['id'])
        #проверяем, есть ли результаты выполнения запроса 
        if len(results_df.columns) > 0:
            results_df = results_df.reindex(columns=['movieLabel.value', 'year.value'], fill_value='no data')
            display(HTML(results_df[['movieLabel.value', 'year.value']].to_html()))
        else:
            print("Нет результатов\n")
    else:
        print("Фильм не найден\n")

Список фильмов, которые получили Оскар или награду на Каннском фестивале в том же году, что и заданный фильм
World of Apu, The (Apur Sansar) (1959):
Фильм не найден

Faust (1994):
Q29478
Нет результатов

Pather Panchali (1955):
Q622380
Нет результатов

Dead Man (1995):
Q547189


,movieLabel.value,year.value
0,Fargo,1996
1,Independence Day,1996
2,The English Patient,1996
3,Breaking the Waves,1996
4,When We Were Kings,1996
5,Braveheart,1996
6,Secrets & Lies,1996


Titanic (1997):
Q44578


,movieLabel.value,year.value
0,Life is Beautiful,1998
1,Eternity and a Day,1998
2,Titanic,1998
3,The Last Days,1998
